In [2]:
import pandas as pd

# Load each CSV file into a separate DataFrame
file_path1 = r"D:\Machine translation model\Summerization model\Random sampling of test data splits\randomsampling bengali\urls_and_articles_summary_bengali.csv"
file_path2 = r"D:\Machine translation model\Summerization model\Random sampling of test data splits\randomsampling gujarati\urls_and_articles_summary_gujarati.csv"
file_path3 = r"D:\Machine translation model\Summerization model\Random sampling of test data splits\randomsampling hindi\urls_and_articles_summary_hindi.csv"
file_path4 = r"D:\Machine translation model\Summerization model\Random sampling of test data splits\randomsampling marathi\urls_and_articles_summary_marathi.csv"
file_path5 = r"D:\Machine translation model\Summerization model\Random sampling of test data splits\randomsampling tamil\urls_and_articles_summary_tamil.csv"
file_path6 = r"D:\Machine translation model\Summerization model\Random sampling of test data splits\randomsampling telugu\urls_and_articles_summary_telugu.csv"

# Load each file and store in a separate DataFrame
df1 = pd.read_csv(file_path1)
df2 = pd.read_csv(file_path2)
df3 = pd.read_csv(file_path3)
df4 = pd.read_csv(file_path4)
df5 = pd.read_csv(file_path5)
df6 = pd.read_csv(file_path6)

# Preview the data for each file to confirm the "Article" column is loaded correctly
print("File 1 Preview:")
print(df1.head())

print("File 2 Preview:")
print(df2.head())

print("File 3 Preview:")
print(df3.head())

print("File 4 Preview:")
print(df4.head())

print("File 5 Preview:")
print(df5.head())

print("File 6 Preview:")
print(df6.head())



File 1 Preview:
                                  URL Source  \
0  https://www.bbc.com/bengali/news-56713767   
1  https://www.bbc.com/bengali/news-56533637   
2  https://www.bbc.com/bengali/news-56631353   
3  https://www.bbc.com/bengali/news-47189627   
4  https://www.bbc.com/bengali/news-43949363   

                                             Article  \
0  বাংলাদেশের ইসলামিক ফাউন্ডেশনও গত ১৪ই মার্চ ইসল...   
1  বিক্ষোভের সময় পুলিশ কাঁদানে গ্যাস ও রাবার বুল...   
2  ডুবে যাওয়া লঞ্চটি মুন্সিগঞ্জ থেকে নারায়ণগঞ্জ...   
3  ২০১৮ সালের নির্বাচন প্রত্যাখ্যান করেছে বিএনপি।...   
4  কার্টুনে দাম্পত্য জীবন মহারাষ্ট্রের কোলাপুরের ...   

                                             Summary  
0  ইসলামী শিক্ষাবিদ এবং যুক্তরাজ্যের জাতীয় স্বাস...  
1  বাংলাদেশে ভারতীয় প্রধানমন্ত্রী নরেন্দ্র মোদীর...  
2  বাংলাদেশে রাজধানী ঢাকার নিকটবর্তী নারায়ণগঞ্জে...  
3  বাংলাদেশে গত ৩০শে ডিসেম্বর অনুষ্ঠিত সংসদ নির্ব...  
4  বিয়ে টিকিয়ে রাখার ক্ষেত্রে স্বামী ও স্ত্রীর ...  
File 2 Preview:
            

In [ ]:
import cohere
import pandas as pd
from rouge import Rouge
import re
from concurrent.futures import ThreadPoolExecutor, as_completed

# Set up your Cohere API key
co = cohere.Client('API KEY') 

def clean_text(text):
   
    return re.sub(r'�', '', text)


def summarize_article(article):
    try:
        # Cohere's API call to generate a summary
        response = co.generate(
            model='command-r-plus',  # Choose the appropriate model size (e.g., 'xlarge' or 'command-xlarge-nightly')
            prompt=f"""
            Summarize the following article briefly in Bengali language.
            Article:
            {article}
            Summary:
            """,
            max_tokens=300,  # Adjust based on the desired summary length
            temperature=0.3,  # Control creativity of the response
            stop_sequences=["Summary:"]
        )
        
        summary = response.generations[0].text.strip()
        return summary
    except Exception as e:
        print(f"Error summarizing article: {e}")
        return None

def process_dataframe(df):
    all_summaries = []
    rouge_scores = []

    # Initialize ROUGE evaluator
    rouge = Rouge()

    # Using ThreadPoolExecutor for parallel summarization
    with ThreadPoolExecutor(max_workers=5) as executor:
        future_to_article = {executor.submit(summarize_article, article): article for article in df['Article']}
        
        for future in as_completed(future_to_article):
            article = future_to_article[future]
            try:
                summary = future.result()
                if summary:  # Check if a valid summary was returned
                    all_summaries.append(summary)

                    # Print the generated summary
                    print(f"Generated Summary for Article: {article[:50]}...")  # Print the first 50 characters of the article
                    print(summary)
                    print("-" * 80)  # Separator for clarity

                    # Calculate ROUGE score if reference summary is available
                    if 'Summary' in df.columns:
                        reference_summary = df.loc[df['Article'] == article, 'Summary'].values[0]
                        score = rouge.get_scores(summary, reference_summary, avg=True)
                        rouge_scores.append(score)
                        print(f"ROUGE Score for the summary: {score}")
                    else:
                        rouge_scores.append(None)

                else:
                    all_summaries.append(None)
                    rouge_scores.append(None)
            except Exception as e:
                print(f"Error processing article: {e}")
                all_summaries.append(None)
                rouge_scores.append(None)

    return all_summaries, rouge_scores  # Return the generated summaries and ROUGE scores

# Process only df1
print("Processing df1...")
generated_summaries, rouge_scores = process_dataframe(df1)

# Create a DataFrame to store the results
results_df = pd.DataFrame({
    'Article': df1['Article'],  # Original articles
    'Generated_Summary': generated_summaries,  # Generated summaries
    'ROUGE_Score': [str(score) for score in rouge_scores]  # Convert ROUGE scores to string for storage
})

# Save the results to an Excel file
output_file_path = "output_with_summaries_bengali.xlsx"  # Specify your desired output file path
results_df.to_excel(output_file_path, index=False)

print(f"Results saved to {output_file_path}")

Processing df1...
Generated Summary for Article: কার্টুনে দাম্পত্য জীবন মহারাষ্ট্রের কোলাপুরের এক দ...
মুম্বাই হাইকোর্ট এক বিয়েকে অর্থহীন ঘোষণা করেছে, কারণ বিবাহিত দম্পতির মধ্যে কোনও শারীরিক সম্পর্ক ছিল না, যদিও স্বামী এমন দাবি করেছিলেন। বিয়ে হয়েছিল ২০০৯ সালে, কিন্তু স্ত্রী বিয়ের ঠিক পরপরই অভিযোগ করেছিলেন যে, তাকে ধোঁকা দেওয়া হয়েছে। বিচারপতি মনে করেছেন য
--------------------------------------------------------------------------------
ROUGE Score for the summary: {'rouge-1': {'r': 0.21739130434782608, 'p': 0.12195121951219512, 'f': 0.15624999539550796}, 'rouge-2': {'r': 0.045454545454545456, 'p': 0.025, 'f': 0.03225805993756569}, 'rouge-l': {'r': 0.13043478260869565, 'p': 0.07317073170731707, 'f': 0.09374999539550805}}
Generated Summary for Article: বাংলাদেশের ইসলামিক ফাউন্ডেশনও গত ১৪ই মার্চ ইসলামিক...
ইসলামিক ফাউন্ডেশন এবং দেশের জ্যেষ্ঠ আলেমরা বলছেন, রমজানের সময় রোজা রেখে করোনাভাইরাসের টিকা নেয়ার কোনও সমস্যা নেই। কারণ, টিকা মাংসপেশিতে দেয়া হয় এবং তা খাদ্যনালীতে প্রবেশ করে না।

In [5]:
import cohere
import pandas as pd
from rouge import Rouge
import re
from concurrent.futures import ThreadPoolExecutor, as_completed

# Set up your Cohere API key
co = cohere.Client('API KEY') 

def clean_text(text):
   
    return re.sub(r'�', '', text)


def summarize_article(article):
    try:
        # Cohere's API call to generate a summary
        response = co.generate(
            model='command-r-plus',  # Choose the appropriate model size (e.g., 'xlarge' or 'command-xlarge-nightly')
            prompt=f"""
            Summarize the following article briefly in Gujarati language.
            Article:
            {article}
            Summary:
            """,
            max_tokens=300,  # Adjust based on the desired summary length
            temperature=0.3,  # Control creativity of the response
            stop_sequences=["Summary:"]
        )
        
        summary = response.generations[0].text.strip()
        return summary
    except Exception as e:
        print(f"Error summarizing article: {e}")
        return None

def process_dataframe(df):
    all_summaries = []
    rouge_scores = []

    # Initialize ROUGE evaluator
    rouge = Rouge()

    # Using ThreadPoolExecutor for parallel summarization
    with ThreadPoolExecutor(max_workers=5) as executor:
        future_to_article = {executor.submit(summarize_article, article): article for article in df['Article']}
        
        for future in as_completed(future_to_article):
            article = future_to_article[future]
            try:
                summary = future.result()
                if summary:  # Check if a valid summary was returned
                    all_summaries.append(summary)

                    # Print the generated summary
                    print(f"Generated Summary for Article: {article[:50]}...")  # Print the first 50 characters of the article
                    print(summary)
                    print("-" * 80)  # Separator for clarity

                    # Calculate ROUGE score if reference summary is available
                    if 'Summary' in df.columns:
                        reference_summary = df.loc[df['Article'] == article, 'Summary'].values[0]
                        score = rouge.get_scores(summary, reference_summary, avg=True)
                        rouge_scores.append(score)
                        print(f"ROUGE Score for the summary: {score}")
                    else:
                        rouge_scores.append(None)

                else:
                    all_summaries.append(None)
                    rouge_scores.append(None)
            except Exception as e:
                print(f"Error processing article: {e}")
                all_summaries.append(None)
                rouge_scores.append(None)

    return all_summaries, rouge_scores  # Return the generated summaries and ROUGE scores

# Process only df2
print("Processing df2...")
generated_summaries, rouge_scores = process_dataframe(df2)

# Create a DataFrame to store the results
results_df = pd.DataFrame({
    'Article': df2['Article'],  # Original articles
    'Generated_Summary': generated_summaries,  # Generated summaries
    'ROUGE_Score': [str(score) for score in rouge_scores]  # Convert ROUGE scores to string for storage
})

# Save the results to an Excel file
output_file_path = "output_with_summaries_Gujarati.xlsx"  # Specify your desired output file path
results_df.to_excel(output_file_path, index=False)

print(f"Results saved to {output_file_path}")

Processing df2...
Generated Summary for Article: જ્યારે વિરાટ કોહલીની મજબૂત ટીમ તેના જેવી જ મજબૂત અ...
આજે લંડનના ઓવલ મેદાનમાં ભારત અને ઑસ્ટ્રેલિયા વચ્ચે મૅચ રમશે. બંને ટીમની મજબૂતતાનો લાભ ભારતને મળશે. સ્ટીવ સ્મિથ અને ડેવિડ વૉર્નરની ગેરહાજરીમાં ઑસ્ટ્રેલિયાએ સંઘર્ષ કર્યો છે, પરંતુ �
--------------------------------------------------------------------------------
ROUGE Score for the summary: {'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}}
Generated Summary for Article: સામાચાર એજન્સી એએનઆઈ પ્રમાણે આગ લાગવાની આ ઘટના શૉર...
અમદાવાદના ક્લબ્સમાં હોળીની ઉજવણી રદ કરવામાં આવી છે, કારણ કે કોરોનાના કેસોમાં વધારો જાણી રહ્યો છે. ઉત્તર પ્રદેશના ડીઝલના ભાવમાં પેટ્રોલના ભાવોમાં વધારો જોવા મળ્યા છે, જેની અસર મોંઘ�
--------------------------------------------------------------------------------
ROUGE Score for the summary: {'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r

In [10]:
import cohere
import pandas as pd
from rouge import Rouge
from concurrent.futures import ThreadPoolExecutor, as_completed

# Set up your Cohere API key
co = cohere.Client('API KEY') 


def summarize_article(article):
    try:
        # Cohere's API call to generate a summary
        response = co.generate(
            model='command-r-plus',  # Choose the appropriate model size (e.g., 'xlarge' or 'command-xlarge-nightly')
            prompt=f"""
            Summarize the following article briefly in hindi language.
            Article:
            {article}
            Summary:
            """,
            max_tokens=300,  # Adjust based on the desired summary length
            temperature=0.3,  # Control creativity of the response
            stop_sequences=["Summary:"]
        )
        
        summary = response.generations[0].text.strip()
        return summary
    except Exception as e:
        print(f"Error summarizing article: {e}")
        return None

def process_dataframe(df):
    all_summaries = []
    rouge_scores = []

    # Initialize ROUGE evaluator
    rouge = Rouge()

    # Using ThreadPoolExecutor for parallel summarization
    with ThreadPoolExecutor(max_workers=5) as executor:
        future_to_article = {executor.submit(summarize_article, article): article for article in df['Article']}
        
        for future in as_completed(future_to_article):
            article = future_to_article[future]
            try:
                summary = future.result()
                if summary:  # Check if a valid summary was returned
                    all_summaries.append(summary)

                    # Print the generated summary
                    print(f"Generated Summary for Article: {article[:50]}...")  # Print the first 50 characters of the article
                    print(summary)
                    print("-" * 80)  # Separator for clarity

                    # Calculate ROUGE score if reference summary is available
                    if 'Summary' in df.columns:
                        reference_summary = df.loc[df['Article'] == article, 'Summary'].values[0]
                        score = rouge.get_scores(summary, reference_summary, avg=True)
                        rouge_scores.append(score)
                        print(f"ROUGE Score for the summary: {score}")
                    else:
                        rouge_scores.append(None)

                else:
                    all_summaries.append(None)
                    rouge_scores.append(None)
            except Exception as e:
                print(f"Error processing article: {e}")
                all_summaries.append(None)
                rouge_scores.append(None)

    return all_summaries, rouge_scores  # Return the generated summaries and ROUGE scores

# Process only df3
print("Processing df3...")
generated_summaries, rouge_scores = process_dataframe(df3)

# Create a DataFrame to store the results
results_df = pd.DataFrame({
    'Article': df3['Article'],  # Original articles
    'Generated_Summary': generated_summaries,  # Generated summaries
    'ROUGE_Score': [str(score) for score in rouge_scores]  # Convert ROUGE scores to string for storage
})

# Save the results to an Excel file
output_file_path = "cohere-output_with_summaries_hindi.xlsx"  # Specify your desired output file path
results_df.to_excel(output_file_path, index=False)

print(f"Results saved to {output_file_path}")

Processing df3...
Generated Summary for Article: माना जा रहा है कि हाल में अमरीकी राष्ट्रपति चुने ग...
न्यूयॉर्क फ़ेडरल रिज़र्व बैंक के अध्यक्ष टिमुथी गाइट्नर को अमेरिकी राष्ट्रपति बराक ओबामा की टीम में वित्त मंत्री बनाए जाने की संभावना के कारण शेयर बाज़ार में तेज़ी आई. एशिया-प्रशांत देशों के सम्मेलन से पहले, राष्ट्रपति जॉर्ज बुश ने चीनी राष्ट्रपति हू जिंताओ से अंतर्राष्ट्रीय वित्तीय संकट पर चर्चा की. पर्यवेक्षकों का मानना है कि बुश एशियाई और अमेरिकी नेताओं पर 20 बड़ी अर्थव्यवस्थाओं द्वारा लिए गए आर्थिक सुधारों के फ़ैसले का समर्थन करन
--------------------------------------------------------------------------------
ROUGE Score for the summary: {'rouge-1': {'r': 0.34782608695652173, 'p': 0.13114754098360656, 'f': 0.19047618649943318}, 'rouge-2': {'r': 0.043478260869565216, 'p': 0.0136986301369863, 'f': 0.020833329689670773}, 'rouge-l': {'r': 0.30434782608695654, 'p': 0.11475409836065574, 'f': 0.1666666626899094}}
Generated Summary for Article: दोनों ने भारत के वीनू मनकड और पंकज राय का 52

In [4]:
import cohere
import pandas as pd
from rouge import Rouge
from concurrent.futures import ThreadPoolExecutor, as_completed

# Set up your Cohere API key
co = cohere.Client('API KEY') 


def summarize_article(article):
    try:
        # Cohere's API call to generate a summary
        response = co.generate(
            model='command-r-plus',  # Choose the appropriate model size (e.g., 'xlarge' or 'command-xlarge-nightly')
            prompt=f"""
            Summarize the following article briefly in marathi language.
            Article:
            {article}
            Summary:
            """,
            max_tokens=300,  # Adjust based on the desired summary length
            temperature=0.3,  # Control creativity of the response
            stop_sequences=["Summary:"]
        )
        
        summary = response.generations[0].text.strip()
        return summary
    except Exception as e:
        print(f"Error summarizing article: {e}")
        return None

def process_dataframe(df):
    all_summaries = []
    rouge_scores = []

    # Initialize ROUGE evaluator
    rouge = Rouge()

    # Using ThreadPoolExecutor for parallel summarization
    with ThreadPoolExecutor(max_workers=5) as executor:
        future_to_article = {executor.submit(summarize_article, article): article for article in df['Article']}
        
        for future in as_completed(future_to_article):
            article = future_to_article[future]
            try:
                summary = future.result()
                if summary:  # Check if a valid summary was returned
                    all_summaries.append(summary)

                    # Print the generated summary
                    print(f"Generated Summary for Article: {article[:50]}...")  # Print the first 50 characters of the article
                    print(summary)
                    print("-" * 80)  # Separator for clarity

                    # Calculate ROUGE score if reference summary is available
                    if 'Summary' in df.columns:
                        reference_summary = df.loc[df['Article'] == article, 'Summary'].values[0]
                        score = rouge.get_scores(summary, reference_summary, avg=True)
                        rouge_scores.append(score)
                        print(f"ROUGE Score for the summary: {score}")
                    else:
                        rouge_scores.append(None)

                else:
                    all_summaries.append(None)
                    rouge_scores.append(None)
            except Exception as e:
                print(f"Error processing article: {e}")
                all_summaries.append(None)
                rouge_scores.append(None)

    return all_summaries, rouge_scores  # Return the generated summaries and ROUGE scores

# Process only df4
print("Processing df4...")
generated_summaries, rouge_scores = process_dataframe(df4)

# Create a DataFrame to store the results
results_df = pd.DataFrame({
    'Article': df4['Article'],  # Original articles
    'Generated_Summary': generated_summaries,  # Generated summaries
    'ROUGE_Score': [str(score) for score in rouge_scores]  # Convert ROUGE scores to string for storage
})

# Save the results to an Excel file
output_file_path = "cohere-output_with_summaries_marathi.xlsx"  # Specify your desired output file path
results_df.to_excel(output_file_path, index=False)

print(f"Results saved to {output_file_path}")

Processing df4...
Generated Summary for Article: संगीतकार अजय अतुल यांचा भारतातल्या 100 सेलिब्रिटीं...
संगीतकार अजय-अतुल यांनी वर्षाच्या भारतातील 100 सेलिब्रिटींच्या यादीत 22 व्या क्रमांकावर स्थान गेपवून बॉलिवुडमध्ये हे वर्ष गाजवलं. या यादीत माधुरी दिक्षीत, शंकर-एहसान-लॉय, शाहिद कपूर, करण जोहर यांचाही समावेश आहे. या वर्षी यादीत असलेल्या 100 जणांनी गेल्या वर्षीच्या यादीतील 100 जणांच्या कमाईपेक्षा 22 टक्के जास्त कमाई केली आहे. या यादीत विराट कोहली पहिल्या क्रमांकावर, अक्षय कुमार दुसर्या क्
--------------------------------------------------------------------------------
ROUGE Score for the summary: {'rouge-1': {'r': 0.25, 'p': 0.14583333333333334, 'f': 0.18421052166205}, 'rouge-2': {'r': 0.03333333333333333, 'p': 0.018518518518518517, 'f': 0.02380951921768796}, 'rouge-l': {'r': 0.25, 'p': 0.14583333333333334, 'f': 0.18421052166205}}
Generated Summary for Article: स्टीव्हन स्मिथने वेगवान शतक झळकावलं. सिडनी क्रिकेट...
सिडनीत झालेल्या दुसऱ्या वनडेत ऑस्ट्रेलियाने भारताला 389 रन्सचा लक्ष्य दिल

In [6]:
import cohere
import pandas as pd
from rouge import Rouge
from concurrent.futures import ThreadPoolExecutor, as_completed

# Set up your Cohere API key
co = cohere.Client('API KEY') 


def summarize_article(article):
    try:
        # Cohere's API call to generate a summary
        response = co.generate(
            model='command-r-plus',  # Choose the appropriate model size (e.g., 'xlarge' or 'command-xlarge-nightly')
            prompt=f"""
            Summarize the following article briefly in Tamil language.
            Article:
            {article}
            Summary:
            """,
            max_tokens=300,  # Adjust based on the desired summary length
            temperature=0.3,  # Control creativity of the response
            stop_sequences=["Summary:"]
        )
        
        summary = response.generations[0].text.strip()
        return summary
    except Exception as e:
        print(f"Error summarizing article: {e}")
        return None

def process_dataframe(df):
    all_summaries = []
    rouge_scores = []

    # Initialize ROUGE evaluator
    rouge = Rouge()

    # Using ThreadPoolExecutor for parallel summarization
    with ThreadPoolExecutor(max_workers=5) as executor:
        future_to_article = {executor.submit(summarize_article, article): article for article in df['Article']}
        
        for future in as_completed(future_to_article):
            article = future_to_article[future]
            try:
                summary = future.result()
                if summary:  # Check if a valid summary was returned
                    all_summaries.append(summary)

                    # Print the generated summary
                    print(f"Generated Summary for Article: {article[:50]}...")  # Print the first 50 characters of the article
                    print(summary)
                    print("-" * 80)  # Separator for clarity

                    # Calculate ROUGE score if reference summary is available
                    if 'Summary' in df.columns:
                        reference_summary = df.loc[df['Article'] == article, 'Summary'].values[0]
                        score = rouge.get_scores(summary, reference_summary, avg=True)
                        rouge_scores.append(score)
                        print(f"ROUGE Score for the summary: {score}")
                    else:
                        rouge_scores.append(None)

                else:
                    all_summaries.append(None)
                    rouge_scores.append(None)
            except Exception as e:
                print(f"Error processing article: {e}")
                all_summaries.append(None)
                rouge_scores.append(None)

    return all_summaries, rouge_scores  # Return the generated summaries and ROUGE scores

# Process only df5
print("Processing df5...")
generated_summaries, rouge_scores = process_dataframe(df5)

# Create a DataFrame to store the results
results_df = pd.DataFrame({
    'Article': df5['Article'],  # Original articles
    'Generated_Summary': generated_summaries,  # Generated summaries
    'ROUGE_Score': [str(score) for score in rouge_scores]  # Convert ROUGE scores to string for storage
})

# Save the results to an Excel file
output_file_path = "cohere-output_with_summaries_Tamil.xlsx"  # Specify your desired output file path
results_df.to_excel(output_file_path, index=False)

print(f"Results saved to {output_file_path}")

Processing df5...
Generated Summary for Article: சித்தரிப்புக்காக வெனிசுவேலா நாட்டில் ஏறத்தாழ 50 ஆய...
வெனிசுவேலாவில் சித்தரிப்புக்காக வாக்கு இயந்திரங்கள் மற்றும் கணினிகள் எரிந்து சாம்பலானதால், இந்த ஆண்டு இறுதியில் நடைபெறவிருந்த தேர்தல் தள்ளிப் போகுமா என்பது தெளிவில்லை. இத்தாலியில், கொரோனா வைரஸ் தடுப்பு நடவடிக்கைகளுக்கு எதிராக சிறைச்சாலையில் கலவரம் ஏற்பட்டதில் 6 பேர் உ
--------------------------------------------------------------------------------
ROUGE Score for the summary: {'rouge-1': {'r': 0.3333333333333333, 'p': 0.10344827586206896, 'f': 0.15789473322714692}, 'rouge-2': {'r': 0.125, 'p': 0.03571428571428571, 'f': 0.05555555209876565}, 'rouge-l': {'r': 0.2222222222222222, 'p': 0.06896551724137931, 'f': 0.1052631542797785}}
Generated Summary for Article: சரி. கொரோனா வைரஸ் தொற்றால் பாதிக்கப்பட்ட ஒருவர் கு...
கொரோனா வைரஸ் தொற்றிலிருந்து குணமடையும் நேரம் ஒவ்வொருவரது உடல்நிலையையும், அவர்கள் எடுத்துக்கொள்ளும் சிகிச்சையையும் பொறுத்தது. லேசான அறிகுறிகள் உள்ளவர்கள் விரைவில் குணமடைவார்கள்,

In [8]:
import cohere
import pandas as pd
from rouge import Rouge
from concurrent.futures import ThreadPoolExecutor, as_completed

# Set up your Cohere API key
co = cohere.Client('API KEY') 


def summarize_article(article):
    try:
        # Cohere's API call to generate a summary
        response = co.generate(
            model='command-r-plus',  # Choose the appropriate model size (e.g., 'xlarge' or 'command-xlarge-nightly')
            prompt=f"""
            Summarize the following article briefly in Telugu language.
            Article:
            {article}
            Summary:
            """,
            max_tokens=300,  # Adjust based on the desired summary length
            temperature=0.3,  # Control creativity of the response
            stop_sequences=["Summary:"]
        )
        
        summary = response.generations[0].text.strip()
        return summary
    except Exception as e:
        print(f"Error summarizing article: {e}")
        return None

def process_dataframe(df):
    all_summaries = []
    rouge_scores = []

    # Initialize ROUGE evaluator
    rouge = Rouge()

    # Using ThreadPoolExecutor for parallel summarization
    with ThreadPoolExecutor(max_workers=5) as executor:
        future_to_article = {executor.submit(summarize_article, article): article for article in df['Article']}
        
        for future in as_completed(future_to_article):
            article = future_to_article[future]
            try:
                summary = future.result()
                if summary:  # Check if a valid summary was returned
                    all_summaries.append(summary)

                    # Print the generated summary
                    print(f"Generated Summary for Article: {article[:50]}...")  # Print the first 50 characters of the article
                    print(summary)
                    print("-" * 80)  # Separator for clarity

                    # Calculate ROUGE score if reference summary is available
                    if 'Summary' in df.columns:
                        reference_summary = df.loc[df['Article'] == article, 'Summary'].values[0]
                        score = rouge.get_scores(summary, reference_summary, avg=True)
                        rouge_scores.append(score)
                        print(f"ROUGE Score for the summary: {score}")
                    else:
                        rouge_scores.append(None)

                else:
                    all_summaries.append(None)
                    rouge_scores.append(None)
            except Exception as e:
                print(f"Error processing article: {e}")
                all_summaries.append(None)
                rouge_scores.append(None)

    return all_summaries, rouge_scores  # Return the generated summaries and ROUGE scores

# Process only df6
print("Processing df6...")
generated_summaries, rouge_scores = process_dataframe(df6)

# Create a DataFrame to store the results
results_df = pd.DataFrame({
    'Article': df6['Article'],  # Original articles
    'Generated_Summary': generated_summaries,  # Generated summaries
    'ROUGE_Score': [str(score) for score in rouge_scores]  # Convert ROUGE scores to string for storage
})

# Save the results to an Excel file
output_file_path = "cohere-output_with_summaries_Telugu.xlsx"  # Specify your desired output file path
results_df.to_excel(output_file_path, index=False)

print(f"Results saved to {output_file_path}")

Processing df6...
Generated Summary for Article: పాకిస్తాన్, చైనా, నేపాల్‌లతో భారత సరిహద్దు వివాదాల...
భారత్ సరిహద్దు వివాదాలు పాకిస్తాన్, చైనా, నేపాల్ తరఫున ఉన్నాయి. భారత్ సరిహద్దులను నిర్ణయించడం కష్టం కారణం భౌగోళిక, వాస్తవాధీన, అంతర్జాతీయ సరిహద్దు రేఖల మధ్య తేడాలు ఉన్నాయి. భారత్ చైనా
--------------------------------------------------------------------------------
ROUGE Score for the summary: {'rouge-1': {'r': 0.08333333333333333, 'p': 0.10526315789473684, 'f': 0.09302325088155787}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.08333333333333333, 'p': 0.10526315789473684, 'f': 0.09302325088155787}}
Generated Summary for Article: హత్యకు గురయిన విలేకరి కాతా సత్యనారాయణ నెల్లూరులో స...
తుని నియోజకవర్గంలో విలేకరి కాతా సత్యనారాయణ హత్యకు గురయ్యారు. ఇది పాలక పార్టీ నేతలు, ప్రజాప్రతినిధులు ఉన్నారన్న ఆరోపణలను కేసులో నిర్లక్ష్యంగా వ్యవహరించిన పోలీసుల వైఫల్యంతో జరిగిం
--------------------------------------------------------------------------------
ROUGE Score for the summary: {'ro